In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -q --upgrade efficientnet tensorflow_addons

In [ ]:
import math # 數學套件
import re # regular expression
import random 
import os
import itertools #迭代工具
import numpy as np
import pandas as pd
import matplotlib
import scipy
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
import efficientnet.tfkeras as efn
import efficientnet
import sklearn

from matplotlib import pyplot as plt
from datetime import datetime


#checking the packages version
print(f'Numpy version : {np.__version__}')
print(f'Tensorflow version : {tf.__version__}')
print(f'Tensorflow Addons version : {tfa.__version__}')
print(f'EfficientNet (library) version : {efficientnet.__version__}')
print(f'Matplotlib version : {matplotlib.__version__}')
print(f'Scipy version : {scipy.__version__}')
print(f'Pandas version : {pd.__version__}')
print(f'Scikit-Learn version : {sklearn.__version__}')

# 環境設置 
# Set seed and Preconfig

In [ ]:
PRE_TRaining_TIME_START = datetime.now()
AUTO = tf.data.experimental.AUTOTUNE

SEED = 42

os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)

os.environ['TF_DETERMINSTIC_OPS'] = str(SEED)
tf.random.set_seed(SEED)

## Detect TPU Availability

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Runnign on TPU', tup.master())
except ValueError:
    tpu = None
    
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strtegy in Tesorflow. Works on CPU and sigle GPU
    
print('REPLICAS: ', strategy.num_replicas_in_sync)

## Configuration

In [ ]:
from kaggle_datasets import KaggleDatasets
GCS_DS_PATH = KaggleDatasets().get_gcs_path()

In [ ]:
!gsutil ls $GCS_DS_PATH

In [ ]:
IMAGE_SIZE = [224, 224]
EPOCHS = 12
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

In [ ]:
# GCS_PATCH_SELECT = {
#     PD: GCS_DS_PATH + '/_DA_Product_Detection_'
# }
# GCS_PATH = GCS_PATH_SELECT[IMAGE_SIZE[0]]

TRAINING_FILENAMES = tf.io.gfile.glob( '/_DA_Product_Detection_/train/*.tfrecords')

print(len(TRAINING_FILENAMES))

In [ ]:
from kaggle_datasets import KaggleDatasets
IMAGE_SIZE = (512, 512)

GCS_TRAIN_PATHS = [
    KaggleDatasets().get_gcs_path('../input/kaggle/shopee-code-league-20/_DA_Product_Detection_/train'),
#     KaggleDatasets().get_gcs_path('tfrecords-2')
]
TRAINING_FILENAMES = []
for i in GCS_TRAIN_PATHS:
    TRAINING_FILENAMES.append(tf.io.gfile.glob(i + '/*.tfrecords'))
TRAINING_FILENAMES = list(itertools.chain.from_iterable(TRAINING_FILENAMES))

GCS_TEST_PATH = KaggleDatasets().get_gcs_path('tfrecords-3')
TEST_FILENAMES = tf.io.gfile.glob(GCS_TEST_PATH + '/*.tfrecords') # predictions on this dataset should be submitted for the competition

print(len(TRAINING_FILENAMES))
print(len(TEST_FILENAMES))

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
from kaggle_datasets import KaggleDatasets
GCS_PATH = KaggleDatasets().get_gcs_path()

In [ ]:
!gsutil ls $GCS_PATH

In [ ]:
!ls /kaggle/input